##Notebook 2: Entrenamiento del Modelo ##

El objetivo del notebook 2 es entrenar el modelo de análisis de sentimientos.

Aquí se entrenará el modelo Bert para lograr la clasificación de sentimientos con las columnas limpas y procesadas.

Que contiene:

1) Carga del modelo distilbert-base-uncased o xlm-roberta-base para español.
2) Tokenización de textos.
3) Entrenamiento con la calseTrainer de Hugging Face.
4) Guardado y evaluación del modelo.


Importamos librerías

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score


In [ ]:
!pip -q install -U transformers accelerate datasets
import transformers
print("Transformers version:", transformers.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.7 MB/s eta 0:00:00
Transformers version: 4.57.3


In [ ]:
!pip install -q deep-translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00


Montamos Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

Cargar el CSV que está limpio desde Drive

In [ ]:
csv_path = "/content/drive/MyDrive/proyecto_ia/amazon_reviews_limpio.csv"

import pandas as pd
df = pd.read_csv(csv_path)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/proyecto_ia/amazon_reviews_limpio.csv'

Verificamos que está montado correctamente.

In [ ]:
import os
print("¿Existe el archivo?", os.path.exists(csv_path))


¿Existe el archivo? True


1) Verificación rápida del dataset.

In [ ]:
print("Filas y columnas:", df.shape)
print(df["label"].value_counts())
print("Nulos:", df.isna().sum().to_dict())


Filas y columnas: (21255, 2)
label
2    18806
0     2144
1      305
Name: count, dtype: int64
Nulos: {'reviewText': 0, 'label': 0}


2) SpliT Train/Test

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

print("Train:", train_df.shape, "Test:", test_df.shape)


Train: (17004, 2) Test: (4251, 2)


3) Cargar tokenizar + modelo:

Aquí usamos XLM-R

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "xlm-roberta-base"  # recomendado (multilingüe)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4) Dataset de PyTorch para trainer

In [ ]:
import torch

class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.texts = df["reviewText"].tolist()
        self.labels = df["label"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        item["labels"] = torch.tensor(int(self.labels[idx]))
        return item

train_dataset = ReviewsDataset(train_df, tokenizer)
test_dataset  = ReviewsDataset(test_df, tokenizer)

print("Datasets listos ✅", len(train_dataset), len(test_dataset))


Datasets listos ✅ 17004 4251


5) Configurar entrenamiento + métricas

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",        # ✅ en versiones nuevas
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    logging_steps=50
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


6) Entrenar

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.400800,0.395416,0.890849,0.845554


TrainOutput(global_step=2126, training_loss=0.40506430066988813, metrics={'train_runtime': 621.3503, 'train_samples_per_second': 27.366, 'train_steps_per_second': 3.422, 'total_flos': 1118495138761728.0, 'train_loss': 0.40506430066988813, 'epoch': 1.0})

7) Guardar el modelo y tokenizar en Drive:

Esto nos sirve para usar en Api y en el dashboard

In [ ]:
import os

MODEL_DIR = "/content/drive/MyDrive/proyecto_ia/modelos/sentimiento_xlmroberta_v1"
os.makedirs(MODEL_DIR, exist_ok=True)

trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

print("✅ Modelo y tokenizer guardados en:", MODEL_DIR)


✅ Modelo y tokenizer guardados en: /content/drive/MyDrive/proyecto_ia/modelos/sentimiento_xlmroberta_v1


8. Evaluación final y aquí guardamos las métricas.

In [ ]:
import json

eval_metrics = trainer.evaluate()
print("📌 Métricas evaluación:", eval_metrics)

METRICS_PATH = "/content/drive/MyDrive/proyecto_ia/resultados/metrics_sentimiento_v1.json"
os.makedirs(os.path.dirname(METRICS_PATH), exist_ok=True)

with open(METRICS_PATH, "w") as f:
    json.dump(eval_metrics, f, indent=2)

print("✅ Métricas guardadas en:", METRICS_PATH)


📌 Métricas evaluación: {'eval_loss': 0.39541563391685486, 'eval_accuracy': 0.8908492119501293, 'eval_f1': 0.8455544879207669, 'eval_runtime': 32.3706, 'eval_samples_per_second': 131.323, 'eval_steps_per_second': 16.435, 'epoch': 1.0}
✅ Métricas guardadas en: /content/drive/MyDrive/proyecto_ia/resultados/metrics_sentimiento_v1.json


9) En este numeral realizamos el demo de predicción.

In [ ]:
import torch
import torch.nn.functional as F
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source="es", target="en")
id2label = {0:"negativo", 1:"neutro", 2:"positivo"}

def predecir_sentimiento(texto: str, umbral_neutro: float = 0.15, mostrar_probs: bool = True):
    # 1) traducir a inglés (porque entrenaste con reseñas en inglés)
    texto_en = translator.translate(texto)

    # 2) tokenizar
    inputs = tokenizer(texto_en, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 3) inferencia
    with torch.no_grad():
        logits = model(**inputs).logits

    probs = F.softmax(logits, dim=-1).squeeze().cpu().numpy()  # [p_neg, p_neu, p_pos]
    pred = int(probs.argmax())

    # 4) regla de neutro por "baja confianza"
    orden = probs.argsort()[::-1]            # índices de mayor a menor
    top1, top2 = probs[orden[0]], probs[orden[1]]

    if (top1 - top2) < umbral_neutro:
        pred_final = 1  # neutro
    else:
        pred_final = pred

    if mostrar_probs:
        print("Probabilidades (neg, neu, pos):", probs.round(3))

    return id2label[pred_final]


10. Función con traducción.

In [ ]:
from deep_translator import GoogleTranslator
import torch

translator = GoogleTranslator(source="es", target="en")

def predecir_sentimiento(texto: str):
    # Traducir de español a inglés
    texto_en = translator.translate(texto)

    # Tokenizar texto traducido
    inputs = tokenizer(
        texto_en,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Inferencia
    with torch.no_grad():
        logits = model(**inputs).logits

    pred = logits.argmax(-1).item()
    return id2label[pred]


11. Celda de pruebas - demo

In [ ]:
pruebas = [
    "El producto llegó roto y nadie responde, pésimo servicio.",
    "Está bien, cumple con lo esperado, nada especial.",
    "Excelente calidad, llegó antes de tiempo, totalmente recomendado."
]

for t in pruebas:
    print(f"Texto: {t}")
    print(f"Sentimiento: {predecir_sentimiento(t, umbral_neutro=0.22, mostrar_probs=True)}\n")


Texto: El producto llegó roto y nadie responde, pésimo servicio.
Probabilidades (neg, neu, pos): [0.54  0.145 0.315]
Sentimiento: negativo

Texto: Está bien, cumple con lo esperado, nada especial.
Probabilidades (neg, neu, pos): [0.069 0.007 0.924]
Sentimiento: positivo

Texto: Excelente calidad, llegó antes de tiempo, totalmente recomendado.
Probabilidades (neg, neu, pos): [0.069 0.007 0.924]
Sentimiento: positivo



Durante las pruebas de inferencia se observa que la clase ‘neutro’ es menos frecuente.
Y es porque se debe a que el modelo fue entrenado con reseñas de Amazon, donde predominan opiniones claramente positivas o negativas.
En los casos evaluados, la probabilidad asignada a la clase positiva supera ampliamente a las demás, por lo que no se activa la regla de baja confianza para clasificar como neutro.
Por esa razón, este comportamiento es coherente con la distribución del dataset y no representa un error del modelo